In [ ]:
import numpy as np
from psrqpy import QueryATNF
from matplotlib import rcParams
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy import units as u
import math

params = {
          "font.size": 15,
          "font.weight": "normal",
          "xtick.major.size": 5,
          "xtick.minor.size": 3,
          "ytick.major.size": 5,
          "ytick.minor.size": 2,
          "xtick.major.width": 2.5,
          "xtick.minor.width": 3,
          "ytick.major.width": 2.5,
          "ytick.minor.width": 3,
          "lines.linewidth": 1.5,
          "lines.markersize": 7,
          "axes.linewidth": 1.4,
          "axes.linewidth": 2.0,
          "legend.loc": "best",
          "text.usetex": False,    
          "xtick.labelsize" : 14,
          "ytick.labelsize" : 14,
          }

import matplotlib
matplotlib.rcParams.update(params)

rcParams['xtick.direction'] = 'out'
rcParams['ytick.direction'] = 'out'

In [ ]:
query = QueryATNF()

sources = QueryATNF(params=['P0', 'P1', 'ASSOC', 'BINARY', 'TYPE', 'BINCOMP'])

In [ ]:
# log-scale grid
x = np.logspace(-3.0, np.log10(2500), num=100, endpoint=True, base=10.0)

In [ ]:
data = np.genfromtxt('all_SNR_assoc.txt', dtype=str)
P0_SNR = data[:,0].astype(float)
P1_SNR = data[:,1].astype(float)

In [ ]:
# From page 62 of the handbook

R = 10 # km. Multiply by 1e5 for cm

alpha_45 = np.sqrt((3 * pow(3e8*1e2, 3))/(8 * pow(np.pi, 2)) * 1e45 / pow(R*1e5, 6) * pow(np.sin(45*u.degree), 2))
print(alpha_45)

# alpha_90 = 3.2e19

alpha_90 = np.sqrt((3 * pow(3e8*1e2, 3))/(8 * pow(np.pi, 2)) * 1e45 / pow(R*1e5, 6) * pow(np.sin(90*u.degree), 2))
print(alpha_90)

In [ ]:
%matplotlib

fig = plt.figure(figsize=(7,9))

# add spin-down energies Edot ans plot it

def Edot(E, x):
    return E/(4 * (np.pi)**2 * x**(-3) * 10**45)

E = 10**30
while E < 10**39:
    plot, = plt.loglog(x, Edot(E, x), ls='--', color='darkgray')
    E *= 1000
    

# add magnetic field B ans plot it

def field(B, x, alpha):
    return pow(B, 2) / (x * pow(alpha, 2))

B = 10**10
while B < 10**17:
    plot, = plt.loglog(x, field(B, x, alpha_90), ls = '--', color='darkgray')
    B *= 10
    
# add critical field line

# B_cr = 4.4*10**13
# plt.loglog(x, field(B_cr, x, alpha_90), ls = '--', color='lime')

# # Plot from Baring and Harding 1998
xx = 7.9e-13 * pow(x, -11/15)
plt.loglog(x, xx, ls='--', color='#e715ab')
    
# add characteristic age and plot it

def age(t, x):
    return x / (2 * t)
T0 = 3600 * 24 * 365.25 * 1000 * 100 

T = T0
while T < T0 * 1e10:
    plot, = plt.loglog(x, age(T, x), ls=':', color='darkgray')
    T *= 100
    
# add FRB deathline

def FRBdeathline(x, alpha):
    B = 6e13/x
    return pow(B, 2) / (x * pow(alpha, 2))

period_min = np.repeat(0.2, 1000)
pdots = np.linspace(FRBdeathline(0.2, alpha_90), 1e-7, 1000)
x_frb = np.logspace(np.log10(0.2), np.log10(200), num=100, endpoint=True, base=10.0)

plt.loglog(period_min,pdots, ls = '--', color = '#17186a')
plt.loglog(x_frb, FRBdeathline(x_frb, alpha_90), ls = '--', color = '#17186a')

    
# #Death line [eq .(9) from Chen & Ruderman 1993]    
C = 3.2e19
plot, = plt.loglog(x, 10**((78 - 7 * np.log10(C) + 9.5 * np.log10(x)) / 3.5), color='k', 
                   ls='-', alpha=0.6)

# #Death line (Ruderman and Southerland)
# ydeath_line=(2.822e-17)*x**3
# plt.loglog(x, ydeath_line, color='k', linestyle='-', alpha=0.6)

#Death line (ZHM equation 4.  -- CR Vacuum gap model)
rho6=1   # radius of NS in units of 10^6 cm.
ydeath_line2=10**((9/4 * np.log10(x))-16.58+np.log10(rho6))
plt.loglog(x, ydeath_line2, color='k', linestyle='-.', alpha=0.6)

#Death line (ZHM equation 9.  -- CR SCLF model)
rho6=1   # radius of NS in units of 10^6 cm.
ydeath_line3=10**((2 * np.log10(x))-16.52+np.log10(rho6))
plt.loglog(x, ydeath_line3, color='k', linestyle='--', alpha=0.6)

#Add the really slow pulsars

plt.text(3, 10**-15.7, 'J2144$-$3933', fontsize=11)
plt.loglog(8.5, 4.9*10**(-16), color="r", marker='*', ms=9)  # 8.5s pulsar

plt.text(15, 10**-14.1, 'J2251$-$3711', fontsize=11)
plt.loglog(12.1, 1.3*10**(-14), color="r", marker='+', ms=8, mew=3)  # 12s pulsar

plt.text(30, 10**-13.7, 'J0250$+$5884', fontsize=11)
plt.loglog(23.5, 2.716*10**(-14), color="r", marker='v', ms=7)  # 23s pulsar

plt.text(160, 10**-12.3, 'AR Scorpii', fontsize=11)
plt.loglog(117, 7.1*10**(-13), color="r", marker='1', ms=12, mew=3)  # AR Sco

plt.text(95, 10**-12.8, 'J0901$-$4046', fontsize=11, weight='bold')
plt.loglog(75.88, 2.254*10**(-13), color="r", marker='X', ms=10, mew=0.1)  # 76s pulsar; Paper value

plt.text(12, 10**-9.5, 'GLEAM-X J162759.5-523504.3', fontsize=11)
plt.loglog(1090.8, 10**(-9.1), color="r", marker=r'$\downarrow$', ms=15, mew=0.1)  # 18.18 min source  


plt.xlim(1e-2, 2000)
plt.ylim(1e-18, 10**(-8.0))

plt.tick_params(axis='y', which='minor')

plt.loglog(sources['P0'], sources['P1'], marker = '.', ls = 'none', 
           color = '#c3d5e7', alpha=0.8, ms=3, label='Radio-loud pulsars')

# Plot RRATs

plt.loglog(sources['P0'][np.where(sources['TYPE']=='RRAT')], 
           sources['P1'][np.where(sources['TYPE']=='RRAT')], color='#04b4d8', marker='o', ls='none', 
           mec='#04b4d8', label='RRAT')

# Plot XINS

plt.loglog(sources['P0'][np.where(sources['TYPE']=='XINS')], 
           sources['P1'][np.where(sources['TYPE']=='XINS')], color='#aad922', marker='d', ls='none',
           mec='#aad922', ms=7, label='XINS')

# Plot Binary

plt.loglog(sources['P0'][np.where(sources['TYPE']=='AXP')], 
           sources['P1'][np.where(sources['TYPE']=='AXP')], color='#b338c2', marker='s', ls='none', 
           alpha=0.9, mec='#b338c2', ms=7, label='Magnetar')

# Plot SNR

plt.loglog(P0_SNR, P1_SNR, color='#1dd3b0', marker='^', ls='none', mec='#1dd3b0', ms=7, label='SNR')

c = '#144e92'

# Plot sources with binary companions

plt.loglog(sources['P0'][np.where(sources['BINCOMP']=='UL')], 
           sources['P1'][np.where(sources['BINCOMP']=='UL')], color=c, marker='>', ls='none', 
           alpha=0.9, mec=c, ms=7, label='Binary')

plt.legend(fontsize=11, loc=2)

plt.loglog(sources['P0'][np.where(sources['BINCOMP']=='MS')], 
           sources['P1'][np.where(sources['BINCOMP']=='MS')], color=c, marker='>', ls='none', 
           alpha=0.9, mec=c, ms=7)

plt.loglog(sources['P0'][np.where(sources['BINCOMP']=='NS')], 
           sources['P1'][np.where(sources['BINCOMP']=='NS')], color=c, marker='>', ls='none', 
           alpha=0.9, mec=c, ms=7)

plt.loglog(sources['P0'][np.where(sources['BINCOMP']=='CO')], 
           sources['P1'][np.where(sources['BINCOMP']=='CO')], color=c, marker='>', ls='none', 
           alpha=0.9, mec=c, ms=7)

plt.loglog(sources['P0'][np.where(sources['BINCOMP']=='He')], 
           sources['P1'][np.where(sources['BINCOMP']=='He')], color=c, marker='>', ls='none', 
           alpha=0.9, mec=c, ms=7)

plt.text(10**-1.93, 10**-14.5, '100 kyr', fontsize=10,rotation=30)
plt.text(10**-1.93, 10**-16.6, '10 Myr', fontsize=10,rotation=30)
# ax.text(-1.5, -16.6, '100 Myr', fontsize=10,rotation=10)

# plt.text(10**1.5, 10**-8.4, '$10^{16}$ G', fontsize=10,rotation=326)
plt.text(10**1.4, 10**-10.6, '$10^{15}$ G', fontsize=10,rotation=326)
plt.text(10**-0.03, 10**-11.2, '$10^{14}$ G', fontsize=10,rotation=326)
plt.text(10**-1.93, 10**-11.3, '$10^{13}$ G', fontsize=10,rotation=326)
plt.text(10**-1.93, 10**-13.3, '$10^{12}$ G', fontsize=10,rotation=326)
plt.text(10**-1.93, 10**-15.3, '$10^{11}$ G', fontsize=10,rotation=326)
plt.text(10**-1.93, 10**-17.3, '$10^{10}$ G', fontsize=10,rotation=326)

plt.text(0.7, 10**-10.4, '$10^{36}$ erg s$^{-1}$', fontsize=10,rotation=64)
plt.text(20, 10**-12, '$10^{30}$ erg s$^{-1}$', fontsize=10,rotation=63)

plt.text(10**-1, 10**-11.2, '$B_\mathrm{cr}$', fontsize=14, rotation=335, color = '#e715ab')

plt.text(10**1.2, 10**-16.4, 'Low-twist deathline', fontsize=11, rotation=294, color = '#17186a')


plt.xlabel('Period (s)', fontsize=13)
plt.ylabel('Period derivative (s s$^{-1}$)', fontsize=13)

plt.tight_layout()

# plt.show()

plt.savefig('PPdot_MTP0013_Edot.png', dpi=1000)